# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

Preparação da planilha de usuários usando a API POST

In [ ]:
# acessa a API disponibilizada no Swagger: https://sdw-2023-prd.up.railway.app/swagger-ui.html
# no método POST é possível criar usuários - na minha aplicação, criei os usuários "Dory", "Briise" e "Manas"
# no método GET /users - all users - é possível visualizar todos os usuários da API cadastrados e atualizados até o momento além de verificar se os usuários recém criados estão na base
# cria um arquivo de texto (SDW2023.csv) com os IDs dos 3 usuários criados e salva

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
#pip install pandas

In [17]:
#TODO Extrair os IDs do arquivo CSV.
#obs: antes, é necessário realizar o upload do arquivo "SDW23.csv" para o Colab

import pandas as pd

df = pd.read_csv('SDW23.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3414, 3415, 3417]


In [18]:
#TODO Obter os dados de cada ID usando a API da Santander Dev Week 2023

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app' #lembrando que será acrescentado "/users/{id}" ao final da consulta GET

In [19]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}') # formato similar a resposta na api get. Ex.: https://sdw-2023-prd.up.railway.app/users/3417 sessão get/users/{id} em teste pra um dos ids criados
  return response.json() if response.status_code == 200 else None #consultar o status code das possíveis respostas da API está demonstrado no swagger

users = [user for id in user_ids if (user := get_user(id)) is not None] #cria uma lista com os users a serem consultados usando list compreenhension e o operador walrus (:=)
print(json.dumps(users, indent=2)) #printa (opcional) a lista users formatando o json usando uma indentação 2

[
  {
    "id": 3414,
    "name": "Dory",
    "account": {
      "id": 3637,
      "number": "1221-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3310,
      "number": "**** **** **** 1208",
      "limit": 2000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3415,
    "name": "Briise",
    "account": {
      "id": 3638,
      "number": "1223-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3311,
      "number": "**** **** **** 2312",
      "limit": 2000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3417,
    "name": "Manas",
    "account": {
      "id": 3640,
      "number": "1225-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3313,
      "number": "**** **** **** 2512",
      "limit": 2000.0
    },
    "features": [],
    "news": []
  }
]


In [ ]:
#caso no csv de consulta seja incluído algum id que não possui cadastro na API a listagem de ids de consulta altera com o numero do id inexistente, porem não retornará resultado correspondente no json print do json da extração.

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.


In [20]:
!pip install openai #instala biblioteca da openAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [33]:
#TODO implementar a integração com o ChatGPT usando o modelo GPT-3.5
#é recomendavel ajustar o texto das roles para conforme a finalidade da campanha
#refinar o prompt para deixar bem direcionado ao público com o mínimo de solicitações possíveis (economia)
import openai

openai.api_key = openai_api_key

def generate_ai_news(user): # usar a referencia de código python obtida no link da api, sessão create chat completion, muda a versão do código para python: https://platform.openai.com/docs/api-reference/chat/create
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system", "content": "Você é um especialista em marketing bancário de investimentos."
      },
      {
          "role": "user", "content": f"Crie uma mensagem utilizando como vocativo o {user['name']} e mencionando sobre a importância dos investimentos (mínimo 50 caracteres e máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"') #retorna a mensagem de retorno do gpt sem aspas

#aplica a função de integração com o chat gpt para todo os usuários referenciados no arquivo
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
      "description": news
  })


Hey, Dory! Não se esqueça: investir é nadar para um futuro financeiramente seguro. #InvistaComSabedoria
Oi, Briise! Não deixe seu dinheiro parado! Invista e conquiste seu futuro financeiro.
E aí, Manas! Você já pensou na importância dos investimentos? Seu futuro financeiro agradece! 💰💪📈


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [35]:
#TODO Atualizar os usuários na API da Santander Dev Week 2023 com os dados enriquecidos
# utilizando a API disponibilizada, opção PUT (update user)

def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"{user['name']} updated? {success}!")

Dory updated? True!
Briise updated? True!
Manas updated? True!


In [41]:
#conferindo o que foi atualizado
print(users)

[{'id': 3414, 'name': 'Dory', 'account': {'id': 3637, 'number': '1221-1', 'agency': '0001', 'balance': 0.0, 'limit': 500.0}, 'card': {'id': 3310, 'number': '**** **** **** 1208', 'limit': 2000.0}, 'features': [], 'news': [{'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg', 'description': 'Hey, Dory! Não esqueça: investir é tão importante quanto nadar! 🐠💰'}, {'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg', 'description': 'Hey, Dory! Não se esqueça: investir é nadar para um futuro financeiramente seguro. #InvistaComSabedoria'}]}, {'id': 3415, 'name': 'Briise', 'account': {'id': 3638, 'number': '1223-1', 'agency': '0001', 'balance': 0.0, 'limit': 500.0}, 'card': {'id': 3311, 'number': '**** **** **** 2312', 'limit': 2000.0}, 'features': [], 'news': [{'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg', 'description': 'Olá, Briise! Descubra como os investi